# OCR del periódico *El Martillo* (Chiclayo, 1903–1919)

Este notebook realiza el proceso de digitalización y estructuración de una página histórica del periódico peruano *El Martillo*, empleando OCR mediante **Google Gemini (API de Google AI Studio)**.
El objetivo es extraer texto, organizarlo en formato tabular y generar una pequeña visualización.

## Flujo del notebook
1. Instalación de librerías
2. Configuración de API Key
3. Descarga de la imagen desde GitHub (RAW URL)
4. OCR con Gemini 1.5 Flash
5. Limpieza y parseo del JSON
6. Exportación del dataset estructurado
7. Visualización simple


In [ ]:
!pip install google-generativeai pandas pillow matplotlib requests

In [ ]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import google.generativeai as genai
import base64
import getpass
import requests
import io

## Configuración de la API Key
Ingresa tu API Key de Google AI Studio. No se verá en pantalla por seguridad.

In [ ]:
api_key = getpass.getpass("Pega tu API Key de Google AI Studio: ")
genai.configure(api_key=api_key)

## Cargar la imagen del periódico desde GitHub (RAW URL)
Esta opción permite no subir archivos a Colab y mantener el vínculo con el repositorio.

In [ ]:
url = "https://raw.githubusercontent.com/ValLovaton/el-martillo-ocr-Pizarro_Sebastian_Lovaton_Valeria/main/data/el_martillo/page_01.png.jpg"

response = requests.get(url)

if response.status_code != 200:
    raise ValueError("No se pudo descargar la imagen desde GitHub. Verifica la URL RAW.")

img = Image.open(io.BytesIO(response.content))
img

## OCR con Google Gemini (modelo 1.5 Flash)
Se solicita un JSON estructurado cumpliendo los requisitos del curso.

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = """
Eres un asistente especializado en OCR y análisis de periódicos históricos.
Se te proporcionará la imagen de una página del periódico peruano *El Martillo* (1916).

Tu tarea es extraer TODO el contenido legible: titulares, artículos, anuncios, secciones
y fragmentos de texto.

Devuelve únicamente una lista en formato JSON. Cada elemento debe contener:
- date
- issue_number
- headline
- section
- type  (article / advertisement / other)
- text_excerpt

No incluyas explicaciones fuera del JSON. El JSON debe ser válido y parseable.
"""

image_bytes = response.content

response_gemini = model.generate_content([
    prompt,
    {"mime_type": "image/png", "data": image_bytes}
])

raw_output = response_gemini.text
raw_output

## Parseo del JSON
Incluye fallback para extraer el JSON incluso si Gemini mezcla texto adicional.

In [ ]:
import re

try:
    structured = json.loads(raw_output)
except:
    matches = re.findall(r"\[.*\]", raw_output, re.DOTALL)
    if matches:
        structured = json.loads(matches[0])
    else:
        raise ValueError("Gemini no devolvió un JSON válido.")

df = pd.DataFrame(structured)
df

## Exportar CSV estructurado
Este archivo deberá subirse a `/data/el_martillo/` en tu repositorio.

In [ ]:
df.to_csv("page_01_structured.csv", index=False)
df.head()

## Visualización simple de los tipos de contenido

In [ ]:
df['type'].value_counts().plot(kind='bar', figsize=(6,4))
plt.title('Distribución de tipos de contenido en la página')
plt.xlabel('Tipo')
plt.ylabel('Cantidad')
plt.show()